<a href="https://colab.research.google.com/github/ds231/linkedin-automation/blob/main/trip_plannerAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required packages
!pip install google-generativeai azure-openai geopy python-dotenv

import google.generativeai as genai
import os
from geopy.geocoders import Nominatim
from datetime import datetime
from IPython.display import clear_output
import time
from datetime import datetime, timedelta

class AIProvider:
    def __init__(self):
        self.provider = None
        self.client = None
        self.setup_successful = False

    def setup_provider(self):
        """Let user choose and setup AI provider"""
        print("Available AI Providers:")
        print("1. Google Gemini (Recommended for free tier)")
        print("2. Microsoft Azure OpenAI")

        while True:
            choice = input("Choose your provider (1 or 2): ")
            if choice == "1":
                return self._setup_gemini()
            elif choice == "2":
                return self._setup_azure()
            else:
                print("Invalid choice. Please enter 1 or 2.")

    def _setup_gemini(self):
        """Setup Google Gemini API"""
        api_key = input("Please enter your Google AI API key: ")
        try:
            genai.configure(api_key=api_key)
            self.model = genai.GenerativeModel('gemini-pro')
            self.provider = "gemini"
            self.setup_successful = True
            clear_output()
            print("Gemini API setup successfully!")
            return True
        except Exception as e:
            print(f"Error setting up Gemini: {str(e)}")
            return False

    def _setup_azure(self):
        """Setup Azure OpenAI"""
        print("\nFor Azure OpenAI, we need the following:")
        api_key = input("Enter Azure OpenAI API key: ")
        endpoint = input("Enter Azure OpenAI endpoint URL: ")
        deployment_name = input("Enter deployment name: ")

        try:
            from openai import AzureOpenAI
            self.client = AzureOpenAI(
                api_key=api_key,
                api_version="2023-05-15",
                azure_endpoint=endpoint
            )
            self.deployment_name = deployment_name
            self.provider = "azure"
            self.setup_successful = True
            clear_output()
            print("Azure OpenAI setup successfully!")
            return True
        except Exception as e:
            print(f"Error setting up Azure OpenAI: {str(e)}")
            return False

    def generate_response(self, prompt):
        """Generate response using selected AI provider"""
        try:
            if self.provider == "gemini":
                response = self.model.generate_content(prompt)
                return True, response.text

            elif self.provider == "azure":
                response = self.client.chat.completions.create(
                    model=self.deployment_name,
                    messages=[
                        {"role": "system", "content": "You are a helpful travel planner assistant."},
                        {"role": "user", "content": prompt}
                    ],
                    max_tokens=1500,
                    temperature=0.7
                )
                return True, response.choices[0].message.content

        except Exception as e:
            return False, f"Error generating response: {str(e)}"

def get_trip_details():
    """Get trip details from user"""
    origin = input("\nWhere are you traveling from?: ")
    destination = input("Where would you like to go?: ")

    while True:
        try:
            start_date = input("When do you want to start your trip? (YYYY-MM-DD): ")
            datetime.strptime(start_date, '%Y-%m-%d')
            break
        except ValueError:
            print("Invalid date format. Please use YYYY-MM-DD")

    while True:
        try:
            duration = int(input("How many days is your trip? (1-14): "))
            if 1 <= duration <= 14:
                break
            print("Please enter a number between 1 and 14")
        except ValueError:
            print("Please enter a valid number")

    while True:
        try:
            budget = int(input("What's your budget in USD?: "))
            if budget > 0:
                break
            print("Budget must be greater than 0")
        except ValueError:
            print("Please enter a valid number")

    return {
        'origin': origin,
        'destination': destination,
        'start_date': start_date,
        'duration': duration,
        'budget': budget
    }

def validate_location(location):
    """Validate if location exists using Geopy"""
    try:
        geolocator = Nominatim(user_agent="my_trip_planner")
        location_data = geolocator.geocode(location)
        return location_data is not None
    except:
        return False

def generate_trip_plan(ai_provider, trip_details):
    """Generate trip plan using selected AI provider"""
    prompt = f"""Create a {trip_details['duration']}-day trip plan from {trip_details['origin']} to {trip_details['destination']}
    starting on {trip_details['start_date']} with a budget of ${trip_details['budget']}.

    Please include:
    1. Daily activities and sightseeing suggestions
    2. Estimated costs for activities
    3. Local transportation tips
    4. Recommended accommodations within budget
    5. Local food recommendations

    Format the response in a clear, day-by-day structure."""

    return ai_provider.generate_response(prompt)

def main():
    print("🌍 Welcome to the AI Trip Planner!")
    print("----------------------------------")

    # Setup AI Provider
    ai_provider = AIProvider()
    if not ai_provider.setup_provider():
        print("Failed to setup AI provider. Please try again.")
        return

    while True:
        # Get trip details
        trip_details = get_trip_details()

        # Validate locations
        if not validate_location(trip_details['destination']):
            print(f"\nCouldn't verify the destination '{trip_details['destination']}'. Please check spelling and try again.")
            continue

        if not validate_location(trip_details['origin']):
            print(f"\nCouldn't verify the origin '{trip_details['origin']}'. Please check spelling and try again.")
            continue

        # Generate and display trip plan
        print("\nGenerating your trip plan... Please wait...")
        success, response = generate_trip_plan(ai_provider, trip_details)

        print("\n📍 Your Trip Plan:")
        print("------------------")
        if success:
            print(response)
        else:
            print(f"Error: {response}")
            print("\nWould you like to try again? The error might be temporary.")

        # Ask if user wants to plan another trip
        another = input("\nWould you like to plan another trip? (yes/no): ").lower()
        if another != 'yes':
            break

    print("\nThank you for using AI Trip Planner! Have a great trip! 🌟")

# Run the program
if __name__ == "__main__":
    main()

Gemini API setup successfully!

Generating your trip plan... Please wait...

📍 Your Trip Plan:
------------------
**Day 1: Arrival in London (12/13/24)**

* Arrive at Heathrow Airport (LHR) and take the Heathrow Express to Paddington Station (£25).
* Check in to a budget-friendly accommodation: Generator Hostel (from £25/night).
* Walk around the city and familiarize yourself with the area.

**Day 2: Tower of London and Westminster Abbey**

* Visit the Tower of London, a historic fortress and castle (£30).
* Take a free tour of Westminster Abbey, the burial place of kings and queens (£20 optional donation).
* Estimated cost: £50

**Day 3: Buckingham Palace and Hyde Park**

* Watch the Changing of the Guard ceremony at Buckingham Palace (free).
* Stroll through Hyde Park and visit the Serpentine Gallery (free).
* Estimated cost: £0

**Day 4: British Museum and Trafalgar Square**

* Explore the British Museum, a world-renowned museum with a vast collection (£12).
* Visit Trafalgar Square